## Population Resource Consumption

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../../az-function/")


import yaml, ssl, asyncio, pickle, os, ast

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

Much of the Population growth is handled in scripts inside the Azure Function

In [2]:
# mapping to the modules that make the app

from app.connectors import cmdb_graph
from app.functions import consumption
from app.functions import growth
from app.objects import time

c = cmdb_graph.CosmosdbClient()


executing local windows deployment
loading syllables from  c:\Users\willi\repos\exoplanets\notebooks\People


In [3]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"app/configurations/popgrowthconfig.yaml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'

Time is updated by the `time` function. It is always running.

In [4]:
t = time.Time(c)
t.get_current_UTU()
params['currentTime'] = t.params['currentTime']
t

< time at UTU:18066 >

In [5]:
params

{'pop_health_requirement': 0.7,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 18066}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

# Calculation which planets will consume

I've given up on trying to get a list of planets with the consumption type. I'm convinced that this could be an Azure Cosmos issue that I can't solve. 

```
g.V().has('label','planet').
    project('planetName', 'grouping').
        by('name').
        by(in('inhabits').out('isOf').as('species').groupCount().by('consumes'))
```

So first we are going to get a list of the locations who have populations. It could be any location, including planets. So I'm going to query based on 

In [6]:
consuming_planets = consumption.get_consuming_planets(c)
consuming_planets

[{'name': 'Damshamunling',
  'label': 'planet',
  'objid': '6135767546801',
  'consumes': [{'organics': 7}]},
 {'name': 'Teinspringsgio',
  'label': 'planet',
  'objid': '9482778660790',
  'consumes': [{'organics': 7}]}]

Then we will create an action event for each pop

In [7]:
message = consumption.get_consumption_message(consuming_planets[0])
message

{'agent': {'name': 'Damshamunling',
  'label': 'planet',
  'objid': '6135767546801',
  'consumes': [{'organics': 7}]},
 'action': 'consume'}

Full Function deployment. 

In [8]:
consumption.calculate_consumption(c,t)

[{'agent': {'name': 'Damshamunling',
   'label': 'planet',
   'objid': '6135767546801',
   'consumes': [{'organics': 7}]},
  'action': 'consume'},
 {'agent': {'name': 'Teinspringsgio',
   'label': 'planet',
   'objid': '9482778660790',
   'consumes': [{'organics': 7}]},
  'action': 'consume'}]

## Consuming



In [9]:
def reduce_location_resource(c,message, consumption):
    # find out if the location has the resource
    objid = message['agent']['objid']
    consuming = list(consumption.keys())[0]
    quantity = list(consumption.values())[0]
    resource_query = f"""
    g.V().has('objid','{objid}').out('has').has('label','resource').has('name','{consuming}').valuemap()
    """
    c.run_query(resource_query)
    if len(c.res) != 1:
        print(f"{objid} has a resource issue - c.res:{c.res}")
    resource = c.clean_nodes(c.res)[0]
    if resource['volume'] > quantity:
        new_volume = resource['volume'] - quantity
        patch_resource_query = f"""
        g.V().has('objid','{objid}').out('has').has('label','resource').has('name','{consuming}')
            .property('volume', {new_volume})
        """
        c.run_query(patch_resource_query)
        print(f"resources on {message['agent']['name']} reduced by {quantity}, {resource['volume']}-> {new_volume}")
    if resource['volume'] <= quantity:
        new_volume = 0
        patch_resource_query = f"""
        g.V().has('objid','{objid}').out('has').has('label','resource').has('name','{consuming}')
            .property('volume', {new_volume})
        """
        c.run_query(patch_resource_query)
        print(f"resources on {message['agent']['name']} reduced by {quantity}, People at this location will starve.")
    return resource


for resource in message['agent']['consumes']:
    # print(message['agent'], resource)
    resource =reduce_location_resource(c,message,resource)
    print(resource)
    

resources on Damshamunling reduced by 7, 286-> 279
{'volume': 286, 'name': 'organics', 'objid': '6439289323440', 'max_volume': 1007, 'description': 'bilogical material that can be consumed by pops', 'replenish_rate': 10, 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'resource', 'id': '6439289323440'}


# Renewing Resources

In [13]:
renewing_resources_query =f"""
   g.V().has('label','resource').has('replenish_rate').valuemap()
"""

In [14]:
c.run_query(renewing_resources_query)
renewing_resources = c.clean_nodes(c.res)
renewing_resources

[{'replenish_rate': 10,
  'volume': 159,
  'name': 'Organic',
  'objid': '4182507921316',
  'max_volume': 895,
  'description': 'bilogical material that can be consumed by pops',
  'username': 'manderson',
  'objtype': 'resource',
  'id': '4182507921316'},
 {'replenish_rate': 10,
  'volume': 9,
  'name': 'organics',
  'objid': '9706157180627',
  'max_volume': 1095,
  'description': 'bilogical material that can be consumed by pops',
  'username': 'Billmanserver',
  'objtype': 'resource',
  'id': '9706157180627'},
 {'replenish_rate': 10,
  'volume': 525,
  'name': 'organics',
  'objid': '0058029712212',
  'max_volume': 1051,
  'description': 'bilogical material that can be consumed by pops',
  'username': 'BillmanLocal2',
  'objtype': 'resource',
  'id': '0058029712212'},
 {'replenish_rate': 10,
  'name': 'organics',
  'objid': '7450760547046',
  'volume': 850,
  'max_volume': 850,
  'description': 'bilogical material that can be consumed by pops',
  'username': 'william.jeffrey.harding@

In [12]:
renewal_messages = []
for resource in renewing_resources:
    if resource['volume'] <= resource['max_volume']:
        renewal_message = growth.get_renewal_message(resource)
        renewal_messages.append(renewal_message)

renewal_messages

[{'agent': {'replenish_rate': 10,
   'volume': 159,
   'name': 'Organic',
   'objid': '4182507921316',
   'max_volume': 895,
   'description': 'bilogical material that can be consumed by pops',
   'username': 'manderson',
   'objtype': 'resource',
   'id': '4182507921316'},
  'action': 'renew'},
 {'agent': {'replenish_rate': 10,
   'volume': 9,
   'name': 'organics',
   'objid': '9706157180627',
   'max_volume': 1095,
   'description': 'bilogical material that can be consumed by pops',
   'username': 'Billmanserver',
   'objtype': 'resource',
   'id': '9706157180627'},
  'action': 'renew'},
 {'agent': {'replenish_rate': 10,
   'volume': 525,
   'name': 'organics',
   'objid': '0058029712212',
   'max_volume': 1051,
   'description': 'bilogical material that can be consumed by pops',
   'username': 'BillmanLocal2',
   'objtype': 'resource',
   'id': '0058029712212'},
  'action': 'renew'},
 {'agent': {'replenish_rate': 10,
   'name': 'organics',
   'objid': '7450760547046',
   'volume': 